In [9]:
import math
import random
import time
from graph.graph import read_graph_from_csv

In [10]:
graph_file = '../graphs/dash/dash.csv'

In [11]:
m = 10 # number of tasks
n = 100 # number of candidate workers

initialize workers by their ids

In [12]:
from stable_matching.TaskWorker import Worker

g = read_graph_from_csv(graph_file,0)
workers = []
worker_ids = random.sample(list(g.nodes.keys()),n)
for i in range(n):
    workers.append(Worker(idx = worker_ids[i]))
del g

initialize tasks : (ids, budget, reverse reachable set, Q)

In [13]:
from graph.QIM import sampling
from utils.funcs import max_k
from stable_matching.TaskWorker import Task
import scipy.stats as stats

mu = 1.0
sigma = 0.1

tasks = []
graph_ids = random.sample(range(100),m)

# allocate budget to every task, sum of all budget is total_budget
avg_budget  = 10
total_budget = avg_budget * m
points = [0]+sorted(random.sample(range(1,total_budget),m-1))+[total_budget]
budgets = [points[i+1]-points[i] for i in range(len(points)-1)]

for i in range(m):
    G = read_graph_from_csv(graph_file,graph_ids[i])
    
    # generate cost of all candidate worker, task cost is generate from a trunc gaussian distribution mu = 1.0, sigma = 0.1, lb = 0.5, ub = 1.5
    X = stats.truncnorm((0.5-mu)/sigma, (1.5-mu)/sigma, loc= mu, scale= sigma).rvs(n)
    costs = {}
    for j in range(n):
        costs[workers[j]] = X[j]
        
    # values of all workers
    values = {}
    Q  = 0.0
    for v in G.nodes:
        values[v] = random.uniform(0.0, 1.0)
        Q += values[v]
        
    budget = budgets[i]
    print('budget:{}'.format(budget))
    start = time.time()
    # generate hyper graph of reverse reachable set in graph G
    k = max_k(budget, costs)
    print('k:{}'.format(k))
    RR = sampling(graph=G, C=worker_ids, k=k, delta=1/n, epsilon=0.01, values=values)
    print('time to generate RR:{}'.format(time.time()-start))
    
    #initialize tasks
    tasks.append(Task(idx=i, budget=budget, R=RR, Q=Q))
    tasks[i].initialize(costs)
    tasks[i].set_choice_max_cover()

budget:10
k:11
i_max:35
time to generate RR:0.891887903213501
budget:7
k:8
i_max:35
time to generate RR:3.3120219707489014
budget:24
k:27
i_max:33
time to generate RR:0.6801424026489258
budget:10
k:11
i_max:35
time to generate RR:1.8207504749298096
budget:11
k:13
i_max:34
time to generate RR:2.11824631690979
budget:5
k:6
i_max:35
time to generate RR:6.105280637741089
budget:10
k:12
i_max:35
time to generate RR:4.042755365371704
budget:14
k:16
i_max:34
time to generate RR:1.364670753479004
budget:4
k:4
i_max:36
time to generate RR:11.171693086624146
budget:5
k:6
i_max:35
time to generate RR:1.7393584251403809


In [14]:
for worker in workers:
    value_dict = {}
    for t in tasks:
        value_dict[t] = random.random()
    worker.set_preference(value_dict)

In [15]:
from stable_matching.stableMatching import generalized_da
for t in tasks:
    t.refresh()
for w in workers:
    w.refresh()
generalized_da(tasks,workers)

In [16]:
result_dict = {}

In [17]:
from stable_matching.TaskWorker import outward_satisfactory
result_dict['outward_satisfactory'] = outward_satisfactory(tasks,workers)

In [18]:
from stable_matching.TaskWorker import individual_rationality_tasks
result_dict['individual_rationality'] = individual_rationality_tasks(tasks)

In [19]:
from stable_matching.TaskWorker import gamma_workers
Sum = 0
Max = 0
Min = 1000
for task in tasks:
    q =  task.Q * gamma_workers(task.R,task.students())/len(task.R)/task.budget 
    Sum += q
    if q> Max:
        Max = q
    if q< Min:
        Min = q
result_dict['average_density'] = Sum/len(tasks)
result_dict['maximum_density'] = Max
result_dict['minimum_density'] = Min

In [20]:
result_dict

{'outward_satisfactory': 0.9489795918367347,
 'individual_rationality': [[10, 9.536486647802905],
  [7, 6.583418247620438],
  [24, 23.859772773309313],
  [10, 8.934723303659581],
  [11, 10.882985740302798],
  [5, 4.849268677448115],
  [10, 9.596755462770957],
  [14, 13.923201745122213],
  [4, 3.268803617317556],
  [5, 4.782096358298838]],
 'average_density': 1.7796835189486249,
 'maximum_density': 2.723589152695055,
 'minimum_density': 0.7870995794112239}